This file addresses to the issues raised in the comments on my last assignment 5 submission.

<pre>
1. "Are any columns related to other columns? (If so, how?) "  

(1) Excluding the column "ACADEMIC_YEAR" since every entry is identical for this variable.
</pre>

In [3]:
import pandas as pd
df = pd.read_csv("earnings_train.csv")
df["ACADEMIC_YEAR"].unique()

array(['2018-2019'], dtype=object)

(2) Excluding the column "DISTRICT_CODE" since it maps to the variable "DISTRICT_NAME" in an one-to-one relationship.

In [3]:
(df.groupby("DISTRICT_CODE")["DISTRICT_NAME"].nunique() == 1).all()

np.True_

(3) Comparing every combinations of the variables to determine their relation.  

(3.1) Given that "DEMO_CATEGORY" has 5 variables, there are 11 - 2 - 2 + 5 = 12 variables to be compared, which means 66 combinations in total.

In [4]:
df["DEMO_CATEGORY"].unique()

array(['Race', 'Homeless Status', 'All', 'Foster Status', 'Gender'],
      dtype=object)

(3.2) Calculating the correlations between the numeric variables (After this step, 66 - 6 = 60 combinations left)  

The wages of all four years are positively correlated to each other

In [5]:
numeric_cols = ["WAGE_YEAR1", "WAGE_YEAR2", "WAGE_YEAR3", "WAGE_YEAR4"]
df[numeric_cols].corr()

,WAGE_YEAR1,WAGE_YEAR2,WAGE_YEAR3,WAGE_YEAR4
WAGE_YEAR1,1.000000,0.990263,0.986502,0.981711
WAGE_YEAR2,0.990263,1.000000,0.992555,0.987440
WAGE_YEAR3,0.986502,0.992555,1.000000,0.993653
WAGE_YEAR4,0.981711,0.987440,0.993653,1.000000


(3.3) Addressing the associations between the combinations involving "DISTRICT_TYPE" (60 - 11 = 49 combinations left)  
<pre>
DISTRICT_TYPE vs DISTRICT_NAME:     N/A (every district name has a fixed corresponding district type, this computation is meaningless)
DISTRICT_TYPE vs Race:              significant correlation, p = 1.3039797078111637e-23 < 0.05
DISTRICT_TYPE vs Homeless Status:   significant correlation, p = 0.002535681334933786 < 0.05
DISTRICT_TYPE vs All:               N/A (the computation is meaningless since "All" has only one value, namely "All")
DISTRICT_TYPE vs Foster Status:     significant correlation, p = 0.0004758717110401368 < 0.05
DISTRICT_TYPE vs Gender:            non-significant, p = 0.8701449669436541 > 0.05
DISTRICT_TYPE vs AWARD_CATEGORY:    non-significant, p = 0.08924945050580084 > 0.05
DISTRICT_TYPE vs WAGE_YEAR1~4:      highly significant, all of the p-values approximate 0.
</pre>

In [6]:
from scipy.stats import chi2_contingency

results = []

 # Addressing the categorical variables using chi square test

for demo_dimension in df["DEMO_CATEGORY"].unique():
    sub_df = df[df["DEMO_CATEGORY"] == demo_dimension].copy()
    table1 = pd.crosstab(sub_df["DISTRICT_TYPE"], sub_df["STUDENT_POPULATION"])
    res1 = chi2_contingency(table1)
    results.append([demo_dimension,res1[1]])

table2 = pd.crosstab(df["DISTRICT_TYPE"], df["AWARD_CATEGORY"])
res2 = chi2_contingency(table2)
results.append(["AWARD_CATEGORY", res2[1]])

# Addressing the numeric variables using Kruskal-Wallis test, given that the numerical variables are not normally distributed

from scipy.stats import kruskal

numeric_vars = ["WAGE_YEAR1", "WAGE_YEAR2", "WAGE_YEAR3", "WAGE_YEAR4"]
for yr in numeric_vars:
    groups = [group[yr] for key, group in df.groupby("DISTRICT_TYPE")]
    p = kruskal(*groups)[1]
    results.append([yr, p])

results

[['Race', np.float64(1.3039797078111637e-23)],
 ['Homeless Status', np.float64(0.002535681334933786)],
 ['All', 1.0],
 ['Foster Status', np.float64(0.0004758717110401368)],
 ['Gender', np.float64(0.8701449669436541)],
 ['AWARD_CATEGORY', np.float64(0.08924945050580084)],
 ['WAGE_YEAR1', np.float64(0.0)],
 ['WAGE_YEAR2', np.float64(0.0)],
 ['WAGE_YEAR3', np.float64(0.0)],
 ['WAGE_YEAR4', np.float64(0.0)]]

(3.4) Addressing the associations between the combinations involving "DISTRICT_NAME" (49 - 10 = 39 combinations left)  
<pre>
DISTRICT_NAME vs every other categorical variable left:  non-significant
DISTRICT_NAME vs WAGE_YEAR1~4:                           significant
</pre>

In [7]:
results2 = []

 # Addressing the categorical variables using chi square test

for demo_dimension in df["DEMO_CATEGORY"].unique():
    sub_df = df[df["DEMO_CATEGORY"] == demo_dimension].copy()
    table1 = pd.crosstab(sub_df["DISTRICT_NAME"], sub_df["STUDENT_POPULATION"])
    res1 = chi2_contingency(table1)
    results2.append([demo_dimension,res1[1]])

table2 = pd.crosstab(df["DISTRICT_NAME"], df["AWARD_CATEGORY"])
res2 = chi2_contingency(table2)
results2.append(["AWARD_CATEGORY", res2[1]])

# Addressing the numeric variables using Kruskal-Wallis test, given that the numerical variables are not normally distributed

numeric_vars = ["WAGE_YEAR1", "WAGE_YEAR2", "WAGE_YEAR3", "WAGE_YEAR4"]
for yr in numeric_vars:
    groups = [group[yr] for key, group in df.groupby("DISTRICT_NAME")]
    p = kruskal(*groups)[1]
    results2.append([yr, p])

results2

[['Race', np.float64(1.0)],
 ['Homeless Status', np.float64(0.9999999999921333)],
 ['All', 1.0],
 ['Foster Status', np.float64(0.9999999999995504)],
 ['Gender', np.float64(1.0)],
 ['AWARD_CATEGORY', np.float64(1.0)],
 ['WAGE_YEAR1', np.float64(0.0)],
 ['WAGE_YEAR2', np.float64(0.0)],
 ['WAGE_YEAR3', np.float64(0.0)],
 ['WAGE_YEAR4', np.float64(0.0)]]

(3.5) Addressing the associations between the combinations involving "AWARD_CATEGORY" (39 - 9 = 30 combinations left)  
<pre>
AWARD_CATEGORY vs Race:              non-significant
AWARD_CATEGORY vs Homeless Status:   non-significant
AWARD_CATEGORY vs All:               N/A (the computation is meaningless since "All" has only one value, namely "All")
AWARD_CATEGORY vs Foster Status:     significant correlation
AWARD_CATEGORY vs Gender:            significant
AWARD_CATEGORY vs WAGE_YEAR1~4:      significant
</pre>

In [8]:
results3 = []

 # Addressing the categorical variables using chi square test

for demo_dimension in df["DEMO_CATEGORY"].unique():
    sub_df = df[df["DEMO_CATEGORY"] == demo_dimension].copy()
    table1 = pd.crosstab(sub_df["AWARD_CATEGORY"], sub_df["STUDENT_POPULATION"])
    res1 = chi2_contingency(table1)
    results3.append([demo_dimension,res1[1]])

# Addressing the numeric variables using Kruskal-Wallis test, given that the numerical variables are not normally distributed

numeric_vars = ["WAGE_YEAR1", "WAGE_YEAR2", "WAGE_YEAR3", "WAGE_YEAR4"]
for yr in numeric_vars:
    groups = [group[yr] for key, group in df.groupby("AWARD_CATEGORY")]
    p = kruskal(*groups)[1]
    results3.append([yr, p])

results3

[['Race', np.float64(8.181939022923938e-10)],
 ['Homeless Status', np.float64(0.051772995563314786)],
 ['All', 1.0],
 ['Foster Status', np.float64(0.015295281739268302)],
 ['Gender', np.float64(0.020135665918818168)],
 ['WAGE_YEAR1', np.float64(0.0)],
 ['WAGE_YEAR2', np.float64(0.0)],
 ['WAGE_YEAR3', np.float64(0.0)],
 ['WAGE_YEAR4', np.float64(0.0)]]

(3.6) Subvariables under the "DEMO_CATEGORY" column.
<pre>
Given that these 5 subvariables are mutually exclusive under the same column in this dataset, we cannot measure their associations (30 - 10 = 20 combinations left).

Race vs WAGE_YEAR1~4:            significant
Homeless Status vs WAGE_YEAR1~4: significant
All:                             N/A
Foster Status vs WAGE_YEAR1~4:   significant
Gender vs WAGE_YEAR1~4:          significant

(20 - 20 = 0 combination left)
</pre>

In [9]:
results4 = []

for demo_dimension in df["DEMO_CATEGORY"].unique():
    sub_df = df[df["DEMO_CATEGORY"] == demo_dimension].copy()
    for yr in numeric_vars:
        groups = [group[yr] for key, group in sub_df.groupby("STUDENT_POPULATION")]
        if len(groups) < 2:
            p = "N/A"
        else:
            p = kruskal(*groups)[1]
        results4.append([demo_dimension, yr, p])

results4

[['Race', 'WAGE_YEAR1', np.float64(0.0)],
 ['Race', 'WAGE_YEAR2', np.float64(0.0)],
 ['Race', 'WAGE_YEAR3', np.float64(0.0)],
 ['Race', 'WAGE_YEAR4', np.float64(0.0)],
 ['Homeless Status', 'WAGE_YEAR1', np.float64(1.295453640923413e-78)],
 ['Homeless Status', 'WAGE_YEAR2', np.float64(6.095880223570833e-79)],
 ['Homeless Status', 'WAGE_YEAR3', np.float64(3.661701762765165e-79)],
 ['Homeless Status', 'WAGE_YEAR4', np.float64(4.929512429386372e-79)],
 ['All', 'WAGE_YEAR1', 'N/A'],
 ['All', 'WAGE_YEAR2', 'N/A'],
 ['All', 'WAGE_YEAR3', 'N/A'],
 ['All', 'WAGE_YEAR4', 'N/A'],
 ['Foster Status', 'WAGE_YEAR1', np.float64(1.1341316494545553e-182)],
 ['Foster Status', 'WAGE_YEAR2', np.float64(1.1059286049586373e-182)],
 ['Foster Status', 'WAGE_YEAR3', np.float64(1.3191024016988204e-182)],
 ['Foster Status', 'WAGE_YEAR4', np.float64(1.2092101364624533e-182)],
 ['Gender', 'WAGE_YEAR1', np.float64(0.3763876263083239)],
 ['Gender', 'WAGE_YEAR2', np.float64(0.34622259684767476)],
 ['Gender', 'WAGE_YEA

<pre>
2. Which demographic shows the highest WAGE_YEAR3? Which demographic shows the lowest WAGE_YEAR3?

(1) The demographic with the highest WAGE_YEAR3: 'Foster Status' - 'Not Foster Youth'
(2) The demographic with the lowest WAGE_YEAR3: 'Race' - 'None Reported'
</pre>

In [4]:
df = pd.read_csv('earnings_train.csv')
result = (df.groupby(["DEMO_CATEGORY","STUDENT_POPULATION"])["WAGE_YEAR3"]
          .mean()
          .reset_index())

sorted_results = result.sort_values("WAGE_YEAR3", ascending=False)

print(sorted_results)

      DEMO_CATEGORY                         STUDENT_POPULATION    WAGE_YEAR3
0               All                                        All  26430.241100
2     Foster Status                           Not Foster Youth  20893.018719
4            Gender                                       Male  20810.238258
3            Gender                                     Female  17612.725000
5   Homeless Status    Did Not Experience Homelessness in K-12  15183.008373
14             Race                                      White  12624.307622
10             Race                         Hispanic or Latino  12535.306968
8              Race                                      Asian   7799.214286
6   Homeless Status           Experienced Homelessness in K-12    944.094429
9              Race                  Black or African American    446.686658
13             Race                          Two or More Races    336.699805
1     Foster Status                               Foster Youth     61.852789

<pre>
3. Are there any people with negative/positive wage trends? Describe these people by their demographics.

    Every demographic group demonstrates a positive wage trend，however, with very different rates. 
    Among all the demographic dimension, foster status predicts the greatest difference in income growth rate.
    The homelessness status also significantly affects a student's income increase rate.
    When race is considered, the groups of White, Hispanic and Asian have noticeably higher increase rate in wages compared with others, 
while the expected increase rates of Black, two or more races, pacific islander, native American and the none reported indicate almost flat growth rate.    
    While Male has a faster growth in wages than the Female, the difference is not as distinguished.

    To summarize, white male who never went through homelessness or have been foster youth have the highest expected income increase rates.

</pre>

In [2]:
from sklearn.linear_model import LinearRegression
import numpy as np

wage_columns = ["WAGE_YEAR1", "WAGE_YEAR2", "WAGE_YEAR3", "WAGE_YEAR4"]
df = df.dropna(subset = wage_columns)
results = []
for (demographic_variable, demographic_value), group in df.groupby(["DEMO_CATEGORY","STUDENT_POPULATION"]):

    # defining the training input to fit the line
    x = np.array([1,2,3,4]).reshape(-1, 1)
    y = group[wage_columns].mean().values.reshape(-1, 1)

    model = LinearRegression().fit(x,y)

    # Find out the slope, and therefore the development trend of the wages.
    slope = model.coef_[0][0]
    
    results.append({"DEMO_CATEGORY":demographic_variable, "STUDENT_POPULATION": demographic_value, "Slope": slope})

results_df = pd.DataFrame(results)
results_df


,DEMO_CATEGORY,STUDENT_POPULATION,Slope
0,All,All,4947.150728
1,Foster Status,Foster Youth,12.287809
2,Foster Status,Not Foster Youth,3770.468571
3,Gender,Female,3352.208839
4,Gender,Male,3599.768232
5,Homeless Status,Did Not Experience Homelessness in K-12,2752.303947
6,Homeless Status,Experienced Homelessness in K-12,171.667120
7,Race,American Indian or Alaska Native,9.055101
8,Race,Asian,1463.797959
9,Race,Black or African American,74.667385
